In [21]:
!pip install psutil

In [22]:
!python -m pip install --upgrade pip

In [23]:
!pip install pyspark

In [1]:
!pip install pyspark

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import findspark
findspark.init("C:\spark")

In [3]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

spark = SparkSession.builder\
    .master('local')\
        .appName('PysparkWordCountApp')\
            .getOrCreate()

sc = spark.sparkContext 

rdd = sc.textFile('leipzig124MB.txt')

print('appName : ', sc.appName)
print('version : ', sc.version)
print('sparkUser : ', sc.sparkUser())

appName :  PysparkWordCountApp
version :  3.4.1
sparkUser :  90530


In [4]:
sc

<SparkContext master=local appName=PysparkWordCountApp>

In [5]:
import socket

host_name = socket.gethostname()
print("Host Name:", host_name)

# IP adresini almak için
ip_address = socket.gethostbyname(host_name)
print("IP Adres:", ip_address)

Host Name: fawern
IP Adres: 172.17.52.118


In [6]:
contractions = {
    r"'m": " am",
    r"’m": ' am',
    r"'s": ' is',
    r"’s": ' is',
    r"isn't": "is not",
    r"Isn't": "Is not",
    r"'re": " are",
    r"’re": " are",
    r"aren't": "are not",
    r"Aren't": "Are not",
    r"'ve": " have",
    r"’ve": " have",
    r"haven't": "have not",
    r"Haven't": "Have not",
    r"'d": " would",
    r"’d": " would",
    r"wouldn't": "would not",
    r"Wouldn't": "Would not",
    r"don't": "do not",
    r"doesn't": "does not",
    r"didn't": "did not",
    r"Don't": "Do not",
    r"Doesn't": "Does not",
    r"Didn't": "Did not",
    r"can't": "cannot",
    r"Can't": "Cannot",
    r"won't": "will not",
    r"Won't": "Will not",
    r"shan't": "shall not",
    r"Shan't": "Shall not",
    r"'ll": " will",
    r"shouldn't": "should not",
    r"Shouldn't": "Should not",
    r"could've": "could have",
    r"Could've": "Could have",
    r"couldn't": "could not",
    r"Couldn't": "Could not",
    r"might've": "might have",
    r"Might've": "Might have",
    r"mustn't": "must not",
    r"Mustn't": "Must not",
}

def remove_contractions(text):
  for pattern, replacement in contractions.items():
    text = re.sub(pattern, replacement, text, flags=re.IGNORECASE)
  return text

def clean_data(x):
  punc='!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~-'
  lowercased_str = x.lower()
  for ch in punc: 
    lowercased_str = lowercased_str.replace(ch, '')
  return lowercased_str


In [7]:
rdd_reCont = rdd.map(remove_contractions)

cleaned_rdd = rdd.map(clean_data)

cleaned_rdd.count() # Number of lines is 1000000

1000000

In [8]:
cleaned_rdd.take(10)

['a rebel statement sent to lisbon from jamba said 86 government soldiers and 13 guerrillas were killed in the fighting that ended jan 3 it said the rebel forces sill held mavinga',
 'authorities last week issued a vacate order for a club in manhattan and closed another in the bronx',
 'at the first pan am bankruptcy hearing for example at least five airlines were represented',
 'mr neigum pokerfaced during the difficult task manages a 46second showing',
 'this combined with the container division talks suggests the groups bankers might be considering an orderly disposal of all assets',
 'she told the post in an interview published sunday that some of the money may have become mingled into improvements on her home that included a swimming pool a 2500 widescreen television and renovations to her basement',
 'according to a study by the marshall institute the average nasa employees age in 1963 was 30 now most of its senior and middlemanagers will be eligible to retire in five years',
 'p

In [9]:
words = cleaned_rdd.flatMap(lambda line : line.split(' '))
words.take(10)

['a',
 'rebel',
 'statement',
 'sent',
 'to',
 'lisbon',
 'from',
 'jamba',
 'said',
 '86']

In [11]:
uniqueWords = words.map(lambda word : (word, 1))
uniqueWords.take(10)

[('a', 1),
 ('rebel', 1),
 ('statement', 1),
 ('sent', 1),
 ('to', 1),
 ('lisbon', 1),
 ('from', 1),
 ('jamba', 1),
 ('said', 1),
 ('86', 1)]

In [12]:
wordCounts = uniqueWords.reduceByKey(lambda x, y : x + y)
wordCounts.take(10)

[('rebel', 1058),
 ('to', 566032),
 ('soldiers', 3413),
 ('and', 448465),
 ('guerrillas', 1950),
 ('were', 72552),
 ('it', 132183),
 ('last', 36100),
 ('closed', 6964),
 ('another', 12331)]

In [13]:
wordCounts.count() # Count of unique words in txt file is 252803

252803

In [39]:
sortedWordCounts = sorted(wordCounts.collect(), key=lambda x : x[1], reverse=True)
sortedWordCounts = spark.sparkContext.parallelize(sortedWordCounts)
sortedWordCounts.take(10)

[('the', 1370260),
 ('of', 595835),
 ('to', 566032),
 ('a', 507283),
 ('in', 473537),
 ('and', 448465),
 ('said', 215897),
 ('for', 214540),
 ('that', 202017),
 ('is', 175742)]

In [40]:
print('Word \t\t\t\t Count')
print('-----------------------------------------------')

counter = 0

for word, count in sortedWordCounts.collect():

    if len(word) <= 3:
        print(f"-{word}   \t\t\t|\t {count}")
    
    elif len(word) > 11:
        print(f"-{word}   \t|\t {count}")

    else:
        print(f"-{word}   \t\t|\t {count}")

    counter += 1
    if counter == 20:
        break

Word 				 Count
-----------------------------------------------
-the   			|	 1370260
-of   			|	 595835
-to   			|	 566032
-a   			|	 507283
-in   			|	 473537
-and   			|	 448465
-said   		|	 215897
-for   			|	 214540
-that   		|	 202017
-is   			|	 175742
-on   			|	 155848
-was   			|	 142318
-it   			|	 132183
-he   			|	 128936
-with   		|	 122650
-by   			|	 121937
-at   			|	 116259
-as   			|	 112893
-from   		|	 107511
-be   			|	 102244


In [41]:
firstTenWords = sortedWordCounts.take(10)

In [42]:
with open("firstTenWord.txt", 'w') as f:
    for word ,count in firstTenWords:
        f.write(f'{word}\t{count}\n')

In [43]:
print('Word \t\t\t\t Count')
print('-----------------------------------------------')

with open('firstTenWord.txt' , 'r') as f:
    lines = f.readlines()

    for line in lines:
        word, count = line.split('\t')
        
        if len(word) <= 3:
            print(f"-{word}   \t\t\t|\t {count}")
        
        elif len(word) > 11:
            print(f"-{word}   \t|\t {count}")

        else:
            print(f"-{word}   \t\t|\t {count}")

Word 				 Count
-----------------------------------------------
-the   			|	 1370260

-of   			|	 595835

-to   			|	 566032

-a   			|	 507283

-in   			|	 473537

-and   			|	 448465

-said   		|	 215897

-for   			|	 214540

-that   		|	 202017

-is   			|	 175742

